In [1]:
from Organizer import Organizer
from search_agent import Research_Tool
import random
LENGTHY_SEARCH_PROMPT = """
                            Answer this question in this format:
                            {"status":"pending", "urls": list(str)}
                            YOU MUST PUT THE JSON ANSWER WITHIN TWO ```

                            Return a list of upto 5 urls from the provided ones that you would like to visit for more Context to answer the following question.
                            Question:
                            
                        """


Researcher = Organizer()
search = Research_Tool()

query = "All available colours of sunflowers"
urls = search.search(query, random.randint(10,15))
print(urls)
data = Researcher.get_filtered_urls(urls, LENGTHY_SEARCH_PROMPT, query)
print(data)
processed_data = await Researcher.process_url(data)
# final_report = await Researcher.multi_text_processor(query)


# final_report

[{'title': 'All Colors Sunflowers - SunflowerSelections.com', 'description': 'Gypsy Charmer explodes with vividcolorsin the garden. Three vibrantcolors, lemon, red, and yellow, are fused into pollenless flowers born on branching plants\xa0...', 'url': 'https://www.sunflowerselections.com/'}, {'title': '13 Crazy-Colorful Sunflowers to Grow in Your Garden', 'description': 'Sunflower colorsrange from cream to gold, yellow, orange, red, mahogany and chocolate brown. There are even pinksunflowers.', 'url': 'https://www.hgtv.com/outdoors/flowers-and-plants/flowers/13-crazy-colorful-sunflowers-pictures'}, {'title': 'Types of Sunflowers: A Guide | Petal Talk', 'description': 'There are several different varieties of bi-color sunflowers. Most feature a dark red center and gradually lighten to yellow at the tip.', 'url': 'https://www.1800flowers.com/articles/flower-facts/infographic-different-types-of-sunflowers'}, {'title': '160 Red & other colors of Sunflowers ideas', 'description': 'Red & oth

[{'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live NowMarried to Real Estate',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live Now100 Day Dream Home',
  'href': 'https://watch.hgtv.com/watch/hgtv?utm_source=marketingsite&utm_medium=livenow'},
 {'text': 'Live Now100 Day Dream Home',
  'href': 'h

In [2]:
import asyncio
from utils import is_article, is_image_url

async def mass_check(list_of_data, function):
    if function == 'image':
        tasks = [is_image_url(i) for i in list_of_data]

    if function == 'link':
        tasks = [is_article(i) for i in list_of_data]

    filtered_data = await asyncio.gather(*tasks)
    return [i for i in filtered_data if i != None]

filtered_images = await mass_check(links, 'link')
print(len(filtered_images))


113


In [3]:
(filtered_images)

[{'text': 'How to Change Hydrangea Color',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/trees-and-shrubs/how-to-change-hydrangea-color'},
 {'text': '18 Lilac Varieties',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/trees-and-shrubs/18-lilac-varieties-pictures'},
 {'text': 'Tall and Tiny Sunflower Varieties',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/flowers/tall-and-tiny-sunflower-varieties-pictures'},
 {'text': 'Growing Sunflowers: When to Plant and How to Grow...',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/flowers/growing-sunflowers-when-to-plant-and-how-to-grow-sunflowers'},
 {'text': '23 Types and Colors of Pretty Poinsettias',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/23-pretty-poinsettias-pictures'},
 {'text': 'Discover 70 Different Daylily Varieties',
  'href': 'https://www.hgtv.com/outdoors/flowers-and-plants/flowers/beautiful-daylily-varieties-pictures'},
 {'text': 'Sunflower Meaning and Symbol

In [2]:
import requests
def is_image_url(url):
    try:
        response = requests.head(url, timeout=5)
        if response.headers["Content-Type"].startswith("image"):
            return url
        else:
            print(response.headers["Content-Type"])
            return None
    except Exception as e:
        print(f"Error checking URL: {e}")
        return None
is_image_url("https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Neymar_Jr._2019.jpg/1200px-Neymar_Jr._2019.jpg")

text/html; charset=utf-8


In [4]:
from utils import fetch_images, fetch_videos

fetch_videos('neymar', 10)

[{'content': 'https://www.youtube.com/watch?v=_NneqSwePwc',
  'description': 'Prime Neymar was one of the most dynamic and exciting players in football history. During his peak years, from around 2014 to 2017, Neymar was widely regarded as one of the top three players in the world, alongside Lionel Messi and Cristiano Ronaldo. His combination of flair, technical skill, and goalscoring ability made him a key figure for ...',
  'duration': '8:35',
  'embed_html': '<iframe width="1280" height="720" src="https://www.youtube.com/embed/_NneqSwePwc?autoplay=1" frameborder="0" allowfullscreen></iframe>',
  'embed_url': 'https://www.youtube.com/embed/_NneqSwePwc?autoplay=1',
  'image_token': '1f792d7b47977d50ee47a2b9215ca34d69967732c1f096d6e6a220768688d3c8',
  'images': {'large': 'https://tse1.mm.bing.net/th?id=OVF.93Ajeh2SQgDZt9JhON66Aw&pid=Api',
   'medium': 'https://tse1.mm.bing.net/th?id=OVF.93Ajeh2SQgDZt9JhON66Aw&pid=Api',
   'motion': '',
   'small': 'https://tse1.mm.bing.net/th?id=OVF.93